In [1]:
import numpy as np
import pandas as pd

In [2]:
#a)
movies = open('hw7_movies-1.txt').read().splitlines()
ids = open('hw7_ids-2.txt').read().splitlines()
ratings = np.loadtxt('hw7_ratings-1.txt', dtype='str')

In [3]:
movie_rating = []
for i in range(len(movies)):
    rec_students = sum(ratings[:,i] == '1')
    seen_students = sum(ratings[:,i] != '?')
    movie_rating.append(rec_students/seen_students)

In [4]:
movie_arr = movies
movie_arr = pd.DataFrame(movie_arr, columns = ['Movie'])
movie_arr['Rating'] = movie_rating
movie_arr.sort_values(by=['Rating']).reset_index(drop=True)

,Movie,Rating
0,Justice League,0.418848
1,The Last Airbender,0.484848
2,Batman v Superman: Dawn of Justice,0.513761
3,Suicide Squad,0.521552
4,Ant-Man and the Wasp,0.579767
5,Solo,0.586207
6,The Shape of Water,0.589041
7,Star Wars: The Last Jedi,0.589744
8,Terminator Genisys,0.603774
9,Wonder Woman,0.609244


In [5]:
#e)
probZ = np.loadtxt('hw7_probZ_init.txt', dtype='float32') 
probR = np.loadtxt('hw7_probR_init.txt', dtype='float32')

In [6]:
k = 4
n_iter = 256  
T = len(ratings)
num_movies = ratings.shape[1]
prob_factor = np.copy(probZ)
prob_movie_given_factor = np.copy(probR)
posterior_probabilities = np.empty([k, T], dtype='float32') 
log_likelihoods = []

In [7]:
prob_factor = np.copy(probZ)
prob_movie_given_factor = np.copy(probR)
posterior_probabilities = np.empty([k, T], dtype='float32') 
log_likelihoods = []

In [8]:
for it in range(n_iter + 1):
    log_likelihood = 0
    temp_prob_factor = np.zeros(k)
    temp_prob_movie_given_factor = np.zeros([num_movies, k])
    
    for user in range(T):
        likelihood = np.zeros(k)
        
        for factor in range(k):
            rec_movies = ratings[user, :] == "1"
            not_rec_movies = ratings[user, :] == "0"
            likelihood[factor] = (prob_factor[factor] * np.prod(prob_movie_given_factor[rec_movies, factor])* np.prod(1 - prob_movie_given_factor[not_rec_movies, factor]))

        total_likelihood = np.sum(likelihood)
        posterior_probabilities[:, user] = likelihood / total_likelihood
        log_likelihood += np.log(total_likelihood)
        
    for factor in range(k):
        temp_prob_factor[factor] = np.mean(posterior_probabilities[factor, :])

        for movie in range(num_movies):
            observed = ratings[:, movie] == "1"
            unobserved = ratings[:, movie] == "?"
            temp_prob_movie_given_factor[movie, factor] = (np.sum(posterior_probabilities[factor, observed]) + prob_movie_given_factor[movie, factor] * np.sum(posterior_probabilities[factor, unobserved])) / np.sum(posterior_probabilities[factor, :])

    log_likelihoods.append(log_likelihood / T)
    prob_factor = temp_prob_factor
    prob_movie_given_factor = temp_prob_movie_given_factor

In [9]:
iterator_arr = [0, 1, 2, 4, 8, 16, 32, 64, 128]
log_likelihood_arr = []
for i in range(len(iterator_arr)):
    log_likelihood_arr.append(log_likelihoods[iterator_arr[i]])

In [10]:
iteration_table = pd.DataFrame(iterator_arr, columns=['Iterations'])
iteration_table['Log Likelihood'] = log_likelihood_arr
iteration_table

,Iterations,Log Likelihood
0,0,-28.089322
1,1,-16.446668
2,2,-14.512415
3,4,-13.568215
4,8,-13.191756
5,16,-13.020406
6,32,-12.976600
7,64,-12.951366
8,128,-12.948197


In [18]:
#f)
user_index = ids.index('A69032048')
movies_not_seen = np.where(ratings[user_index, :] == "?")[0]
expected_ratings = []
for movie_id in movies_not_seen:
    expected_value = np.sum(posterior_probabilities[:, user_index] * prob_factor[:, np.newaxis] * prob_movie_given_factor[movie_id, :])
    expected_ratings.append((movies[movie_id], expected_value))
recommendations = pd.DataFrame(expected_ratings, columns=["Movie", "Predicted Rating"])
sorted_recommendations = recommendations.sort_values(by="Predicted Rating", ascending=False)

In [19]:
sorted_recommendations

,Movie,Predicted Rating
0,The Lord of the Rings: The Fellowship of the Ring,0.879572
2,Ex Machina,0.739327
3,Blade Runner 2049,0.734016
5,Terminator 2,0.727753
6,The Greatest Showman,0.668844
1,Star Wars: The Phantom Menace,0.535050
4,Terminator Genisys,0.533140
7,Solo,0.371904
